in this notebook, I combined the data that is needed for our research project, using 76 countries in GPS as the key because this is our main interest in the project, all the data has been filtered to at least after 2003, for the reason that democratic index only starts from 2006.
notice:
- democratic index range from 2006 to 2019, 2007 & 2009 doesn't exist
- Bosnia Herzegovina have no record in humanitarian aids data.
- 54 countries do not have ODA record
- 63 countries do not have aid received record
 

### data output 
| Variable     |      Content                                                   |
|:-------------|:------------------------------------------------------------- :|
| country| name of countries from Global Preference Survey(GPS) data            |
| isocode | isocode of the countries                             |
|demo2019 ~ demo2006| democratic index by year|
|altruism| altruism score from GPS | 
|gdp2003 ~ gdp2019| gdp in current US dollar by year|
|funding2003 ~ funding2019| humanitarian aid fundinn in US dollar by year       |
|pledge2003 ~ pledge2019|pledging humanitarian aid in US dollar by year        |
|funding2003_gdp ~ funding2019_gdp| funding / gdp the year * 100       |
|gdpcapita2003 ~ gdpcapita2019| gdp per capita in current US dollar by year  |
|region| region group from WDI data|
|income_type| income group from WDI data|
|aid_boolean| True if the country ever receive net official aid between 2003 ~ 2019|
|pop2003 ~ pop2019| population data by year |
|oda2003 ~ oda2019|Net Official development assistance in current US dollar |
|govexpense2003 ~ govexpense2019| General government total expenditure as % of GDP  |
|oda_int| 1 if the country ever receive net official aid between 2003 ~ 2019|
|oecd| dummy variable indicates whether the country belongs to OECD at 2020 |
|g20| dummy variable indicates whether the country belongs to G20 at 2019 |
|gni2003 ~ gni2020| Gini Index data from world bank  |






In [1]:
import pandas as pd
import glob
from auxiliary import *
import zipfile

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)



In [3]:

with zipfile.ZipFile('data.zip', 'r') as zip_ref:
    zip_ref.extractall('')

### read in  democracy index from The Economist's Democracy Index

In [2]:
democracy = pd.read_excel('data/EIU_Democracy_Index_2006_to_2019.xls').rename(columns={"Unnamed: 0": "country"})

In [3]:
country_dict = {'US': 'United States',  'Bosnia and Hercegovina' : 'Bosnia Herzegovina', 'UK' : 'United Kingdom', 'UAE': 'United Arab Emirates' }
democracy = democracy.replace({"country": country_dict})
democracy = democracy.set_index('country').add_prefix('demo').reset_index()
democracy.head()

,country,demo2019,demo2018,demo2017,demo2016,demo2015,demo2014,demo2013,demo2012,demo2011,demo2010,demo2008,demo2006
0,Canada,9.22,9.15,9.15,9.15,9.08,9.08,9.08,9.08,9.08,9.08,9.07,9.07
1,United States,7.96,7.96,7.98,7.98,8.05,8.11,8.11,8.11,8.11,8.18,8.22,8.22
2,Austria,8.29,8.29,8.42,8.41,8.54,8.54,8.48,8.62,8.49,8.49,8.49,8.69
3,Belgium,7.64,7.78,7.78,7.77,7.93,7.93,8.05,8.05,8.05,8.05,8.16,8.15
4,Cyprus,7.59,7.59,7.59,7.65,7.53,7.40,7.29,7.29,7.29,7.29,7.70,7.60


### read in Global Preference Survey (GPS) data and merge with  democracy index 

In [4]:
gps = pd.read_stata('data/country.dta')
gps.shape

(76, 8)

In [5]:
gps.head()

,country,isocode,patience,risktaking,posrecip,negrecip,altruism,trust
0,Afghanistan,AFG,-0.201360,0.120764,0.289641,0.254712,0.166455,0.315964
1,Algeria,DZA,0.059815,0.391530,-0.598255,0.254901,-0.009968,-0.177371
2,Argentina,ARG,-0.229308,0.041503,0.159679,-0.140457,0.027347,-0.206733
3,Australia,AUS,0.657004,0.137137,0.069660,0.022190,0.155331,0.298973
4,Austria,AUT,0.608285,-0.061829,0.161047,-0.055415,-0.035363,0.154628


In [6]:
gps_democracy = pd.merge(gps, democracy, how = 'left', on = 'country')
gps_democracy.shape

(76, 20)

In [7]:
# gps_democracy[gps_democracy.isna().any(axis=1)]
gps_democracy.head()

,country,isocode,patience,risktaking,posrecip,negrecip,altruism,trust,demo2019,demo2018,demo2017,demo2016,demo2015,demo2014,demo2013,demo2012,demo2011,demo2010,demo2008,demo2006
0,Afghanistan,AFG,-0.201360,0.120764,0.289641,0.254712,0.166455,0.315964,2.85,2.97,2.55,2.55,2.77,2.77,2.48,2.48,2.48,2.48,3.02,3.06
1,Algeria,DZA,0.059815,0.391530,-0.598255,0.254901,-0.009968,-0.177371,4.01,3.50,3.56,3.56,3.95,3.83,3.83,3.83,3.44,3.44,3.32,3.17
2,Argentina,ARG,-0.229308,0.041503,0.159679,-0.140457,0.027347,-0.206733,7.02,7.02,6.96,6.96,7.02,6.84,6.84,6.84,6.84,6.84,6.63,6.63
3,Australia,AUS,0.657004,0.137137,0.069660,0.022190,0.155331,0.298973,9.09,9.09,9.09,9.01,9.01,9.01,9.13,9.22,9.22,9.22,9.09,9.09
4,Austria,AUT,0.608285,-0.061829,0.161047,-0.055415,-0.035363,0.154628,8.29,8.29,8.42,8.41,8.54,8.54,8.48,8.62,8.49,8.49,8.49,8.69


### read in UN Humanitarian Affairs Financial Tracking Service data by year and rename columns for merging
### notice Bosnia does not get matched

In [8]:
file_name = 'data/OCHA_FTS_Government_Donations_20'
extension = '.xlsx'
all_filenames = [i for i in glob.glob(f'{file_name}*{extension}')]

In [9]:
for i in range(len(all_filenames)):
    new = pd.read_excel(all_filenames[i], sheet_name='Export data', skiprows=2).rename(columns={"Source org.": "country", "Funding US$": "funding", "Pledges US$" : "pledge"})
    if i == 0:
        df = new
    elif i < 7:
        df = pd.merge(df, new, how='outer', on="country", suffixes=('', f'200{i+3}'))       
    else:
        df = pd.merge(df, new, how='outer', on="country", suffixes=('', f'20{i+3}'))

donate = df.rename(columns={"pledge": "pledge2003", "funding": "funding2003"}).replace({0: np.nan})
donate['country'] = donate['country'].str.rstrip(' Government of')
donate['country'] = donate['country'].str.rstrip(',')


In [10]:
#checking 2006 & 2020 manually 
country_dict = {'United States of America': 'United States',  'Saudi Arabia (Kingdom of)' : 'Saudi Arabia', 'Russian Federation' : 'Russia', 'Korea, Republic of': 'South Korea', 'Viet Nam' : 'Vietnam'}
donate = donate.replace({"country": country_dict})

In [11]:
gps_demo_donate = pd.merge(gps_democracy, donate,  how = 'left', on = 'country')
gps_demo_donate.shape

(76, 56)

In [12]:
gps_demo_donate.head()

,country,isocode,patience,risktaking,posrecip,negrecip,altruism,trust,demo2019,demo2018,demo2017,demo2016,demo2015,demo2014,demo2013,demo2012,demo2011,demo2010,demo2008,demo2006,funding2003,pledge2003,funding2004,pledge2004,funding2005,pledge2005,funding2006,pledge2006,funding2007,pledge2007,funding2008,pledge2008,funding2009,pledge2009,funding2010,pledge2010,funding2011,pledge2011,funding2012,pledge2012,funding2013,pledge2013,funding2014,pledge2014,funding2015,pledge2015,funding2016,pledge2016,funding2017,pledge2017,funding2018,pledge2018,funding2019,pledge2019,funding2020,pledge2020
0,Afghanistan,AFG,-0.201360,0.120764,0.289641,0.254712,0.166455,0.315964,2.85,2.97,2.55,2.55,2.77,2.77,2.48,2.48,2.48,2.48,3.02,3.06,NaN,NaN,NaN,NaN,500000.0,NaN,NaN,NaN,NaN,NaN,24000000.0,NaN,136493.0,NaN,2379910.0,NaN,3360.0,NaN,3000.0,NaN,2000.0,NaN,NaN,NaN,1000.0,NaN,NaN,NaN,NaN,NaN,3870542.0,NaN,903177.0,NaN,NaN,NaN
1,Algeria,DZA,0.059815,0.391530,-0.598255,0.254901,-0.009968,-0.177371,4.01,3.50,3.56,3.56,3.95,3.83,3.83,3.83,3.44,3.44,3.32,3.17,3178000.0,NaN,NaN,NaN,14769749.0,NaN,NaN,NaN,10000.0,NaN,10000.0,NaN,76432.0,NaN,10110696.0,NaN,16010000.0,7000000.0,NaN,NaN,2000000.0,NaN,10000.0,NaN,NaN,NaN,660000.0,NaN,10000.0,NaN,10000.0,NaN,NaN,NaN,NaN,NaN
2,Argentina,ARG,-0.229308,0.041503,0.159679,-0.140457,0.027347,-0.206733,7.02,7.02,6.96,6.96,7.02,6.84,6.84,6.84,6.84,6.84,6.63,6.63,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,139000.0,NaN,31100.0,NaN,50000.0,NaN,107332.0,NaN,130826.0,NaN,165718.0,NaN,230500.0,NaN,NaN,NaN,176272.0,NaN,1344591.0,NaN,265000.0,NaN,155000.0,NaN,NaN,NaN
3,Australia,AUS,0.657004,0.137137,0.069660,0.022190,0.155331,0.298973,9.09,9.09,9.09,9.01,9.01,9.01,9.13,9.22,9.22,9.22,9.09,9.09,87736725.0,NaN,69076907.0,NaN,85073844.0,29859474.0,89757369.0,2262517.0,109371739.0,3817867.0,143674846.0,39494954.0,196311166.0,8733624.0,293856560.0,1923773.0,312711049.0,NaN,271004092.0,2809573.0,263194483.0,NaN,269253261.0,NaN,166236181.0,762236.0,205207886.0,2724017.0,243900706.0,2712522.0,248119692.0,NaN,310639840.0,NaN,271692972.0,NaN
4,Austria,AUT,0.608285,-0.061829,0.161047,-0.055415,-0.035363,0.154628,8.29,8.29,8.42,8.41,8.54,8.54,8.48,8.62,8.49,8.49,8.49,8.69,3245126.0,NaN,6186042.0,NaN,18423937.0,NaN,5624017.0,765306.0,7885069.0,NaN,15280397.0,1017509.0,16939495.0,348675.0,25973202.0,NaN,18074363.0,500409.0,16333100.0,NaN,17699567.0,NaN,18504048.0,NaN,18786627.0,NaN,35663228.0,NaN,49109105.0,NaN,26096928.0,6670328.0,32476153.0,1540154.0,19142494.0,NaN


In [13]:
gps_demo_donate[gps_demo_donate.filter(regex='(funding|pledge)20*', axis=1).isna().all(axis=1)]

,country,isocode,patience,risktaking,posrecip,negrecip,altruism,trust,demo2019,demo2018,demo2017,demo2016,demo2015,demo2014,demo2013,demo2012,demo2011,demo2010,demo2008,demo2006,funding2003,pledge2003,funding2004,pledge2004,funding2005,pledge2005,funding2006,pledge2006,funding2007,pledge2007,funding2008,pledge2008,funding2009,pledge2009,funding2010,pledge2010,funding2011,pledge2011,funding2012,pledge2012,funding2013,pledge2013,funding2014,pledge2014,funding2015,pledge2015,funding2016,pledge2016,funding2017,pledge2017,funding2018,pledge2018,funding2019,pledge2019,funding2020,pledge2020
7,Bosnia Herzegovina,BIH,-0.247258,-0.125639,0.096463,0.429355,-0.11531,-0.193106,4.86,4.98,4.87,4.87,4.83,4.78,5.02,5.11,5.24,5.32,5.7,5.78,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### read in GDP in current US dollar data from WDI and merge with previous data frame

In [14]:
gdp = pd.read_excel('data/GDP_by_country_by_year.xls', sheet_name='Data', skiprows=3).rename(columns={"Country Code": "isocode"}).set_index('isocode')
gdp = gdp.iloc[:,-18 : -1]
gdp = gdp.add_prefix('gdp')
gdp.reset_index()
gdp.head()


,gdp2003,gdp2004,gdp2005,gdp2006,gdp2007,gdp2008,gdp2009,gdp2010,gdp2011,gdp2012,gdp2013,gdp2014,gdp2015,gdp2016,gdp2017,gdp2018,gdp2019
isocode,,,,,,,,,,,,,,,,,
ABW,2.021229e+09,2.228492e+09,2.330726e+09,2.424581e+09,2.615084e+09,2.745251e+09,2.498883e+09,2.390503e+09,2.549721e+09,2.534637e+09,2.701676e+09,2.765363e+09,2.919553e+09,2.965922e+09,3.056425e+09,NaN,NaN
AFG,4.515559e+09,5.226779e+09,6.209138e+09,6.971286e+09,9.747880e+09,1.010923e+10,1.243909e+10,1.585657e+10,1.780429e+10,2.000160e+10,2.056107e+10,2.048489e+10,1.990711e+10,1.936264e+10,2.019176e+10,1.948438e+10,1.910135e+10
AGO,1.781271e+10,2.355205e+10,3.697092e+10,5.238101e+10,6.526645e+10,8.853861e+10,7.030716e+10,8.379950e+10,1.117897e+11,1.280529e+11,1.367099e+11,1.457122e+11,1.161936e+11,1.011239e+11,1.221238e+11,1.013532e+11,9.463542e+10
ALB,5.611492e+09,7.184681e+09,8.052076e+09,8.896074e+09,1.067732e+10,1.288135e+10,1.204422e+10,1.192693e+10,1.289077e+10,1.231983e+10,1.277622e+10,1.322814e+10,1.138685e+10,1.186120e+10,1.301969e+10,1.514702e+10,1.527808e+10
AND,2.361727e+09,2.894922e+09,3.159905e+09,3.456442e+09,3.952601e+09,4.085631e+09,3.674410e+09,3.449967e+09,3.629204e+09,3.188809e+09,3.193704e+09,3.271808e+09,2.789870e+09,2.896679e+09,3.000181e+09,3.218316e+09,3.154058e+09


In [15]:
gps_demo_donate_gdp = pd.merge(gps_demo_donate, gdp, how='left', on="isocode")
gps_demo_donate_gdp.shape

(76, 73)

In [16]:
# gps_demo_donate_gdp[gps_demo_donate_gdp.filter(regex='gdp.*', axis=1).isna().all(axis=1)]
gps_demo_donate_gdp.head()

,country,isocode,patience,risktaking,posrecip,negrecip,altruism,trust,demo2019,demo2018,demo2017,demo2016,demo2015,demo2014,demo2013,demo2012,demo2011,demo2010,demo2008,demo2006,funding2003,pledge2003,funding2004,pledge2004,funding2005,pledge2005,funding2006,pledge2006,funding2007,pledge2007,funding2008,pledge2008,funding2009,pledge2009,funding2010,pledge2010,funding2011,pledge2011,funding2012,pledge2012,funding2013,pledge2013,funding2014,pledge2014,funding2015,pledge2015,funding2016,pledge2016,funding2017,pledge2017,funding2018,pledge2018,funding2019,pledge2019,funding2020,pledge2020,gdp2003,gdp2004,gdp2005,gdp2006,gdp2007,gdp2008,gdp2009,gdp2010,gdp2011,gdp2012,gdp2013,gdp2014,gdp2015,gdp2016,gdp2017,gdp2018,gdp2019
0,Afghanistan,AFG,-0.201360,0.120764,0.289641,0.254712,0.166455,0.315964,2.85,2.97,2.55,2.55,2.77,2.77,2.48,2.48,2.48,2.48,3.02,3.06,NaN,NaN,NaN,NaN,500000.0,NaN,NaN,NaN,NaN,NaN,24000000.0,NaN,136493.0,NaN,2379910.0,NaN,3360.0,NaN,3000.0,NaN,2000.0,NaN,NaN,NaN,1000.0,NaN,NaN,NaN,NaN,NaN,3870542.0,NaN,903177.0,NaN,NaN,NaN,4.515559e+09,5.226779e+09,6.209138e+09,6.971286e+09,9.747880e+09,1.010923e+10,1.243909e+10,1.585657e+10,1.780429e+10,2.000160e+10,2.056107e+10,2.048489e+10,1.990711e+10,1.936264e+10,2.019176e+10,1.948438e+10,1.910135e+10
1,Algeria,DZA,0.059815,0.391530,-0.598255,0.254901,-0.009968,-0.177371,4.01,3.50,3.56,3.56,3.95,3.83,3.83,3.83,3.44,3.44,3.32,3.17,3178000.0,NaN,NaN,NaN,14769749.0,NaN,NaN,NaN,10000.0,NaN,10000.0,NaN,76432.0,NaN,10110696.0,NaN,16010000.0,7000000.0,NaN,NaN,2000000.0,NaN,10000.0,NaN,NaN,NaN,660000.0,NaN,10000.0,NaN,10000.0,NaN,NaN,NaN,NaN,NaN,6.786485e+10,8.532477e+10,1.031984e+11,1.170254e+11,1.349773e+11,1.710006e+11,1.372107e+11,1.612078e+11,2.000195e+11,2.090603e+11,2.097548e+11,2.138088e+11,1.659794e+11,1.600338e+11,1.673894e+11,1.737580e+11,1.699882e+11
2,Argentina,ARG,-0.229308,0.041503,0.159679,-0.140457,0.027347,-0.206733,7.02,7.02,6.96,6.96,7.02,6.84,6.84,6.84,6.84,6.84,6.63,6.63,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,139000.0,NaN,31100.0,NaN,50000.0,NaN,107332.0,NaN,130826.0,NaN,165718.0,NaN,230500.0,NaN,NaN,NaN,176272.0,NaN,1344591.0,NaN,265000.0,NaN,155000.0,NaN,NaN,NaN,1.275870e+11,1.646579e+11,1.987371e+11,2.325573e+11,2.875305e+11,3.615580e+11,3.329765e+11,4.236274e+11,5.301633e+11,5.459824e+11,5.520251e+11,5.263197e+11,5.947493e+11,5.575314e+11,6.426959e+11,5.198715e+11,4.496634e+11
3,Australia,AUS,0.657004,0.137137,0.069660,0.022190,0.155331,0.298973,9.09,9.09,9.09,9.01,9.01,9.01,9.13,9.22,9.22,9.22,9.09,9.09,87736725.0,NaN,69076907.0,NaN,85073844.0,29859474.0,89757369.0,2262517.0,109371739.0,3817867.0,143674846.0,39494954.0,196311166.0,8733624.0,293856560.0,1923773.0,312711049.0,NaN,271004092.0,2809573.0,263194483.0,NaN,269253261.0,NaN,166236181.0,762236.0,205207886.0,2724017.0,243900706.0,2712522.0,248119692.0,NaN,310639840.0,NaN,271692972.0,NaN,4.664881e+11,6.124904e+11,6.934078e+11,7.460542e+11,8.530996e+11,1.053996e+12,9.278052e+11,1.146138e+12,1.396650e+12,1.546152e+12,1.576184e+12,1.467484e+12,1.351694e+12,1.208847e+12,1.330136e+12,1.433904e+12,1.392681e+12
4,Austria,AUT,0.608285,-0.061829,0.161047,-0.055415,-0.035363,0.154628,8.29,8.29,8.42,8.41,8.54,8.54,8.48,8.62,8.49,8.49,8.49,8.69,3245126.0,NaN,6186042.0,NaN,18423937.0,NaN,5624017.0,765306.0,7885069.0,NaN,15280397.0,1017509.0,16939495.0,348675.0,25973202.0,NaN,18074363.0,500409.0,16333100.0,NaN,17699567.0,NaN,18504048.0,NaN,18786627.0,NaN,35663228.0,NaN,49109105.0,NaN,26096928.0,6670328.0,32476153.0,1540154.0,19142494.0,NaN,2.616958e+11,3.009042e+11,3.159744e+11,3.359986e+11,3.886914e+11,4.302943e+11,4.001723e+11,3.918927e+11,4.311203e+11,4.094252e+11,4.300687e+11,4.419961e+11,3.818176e+11,3.952277e+11,4.183162e+11,4.555083e+11,4.463147e+11


### adding columns of funding / GDP * 100 by year

In [17]:
df_funding = convert_percent('funding','gdp', gps_demo_donate_gdp, ratio=100)
df_funding.head()

,country,isocode,patience,risktaking,posrecip,negrecip,altruism,trust,demo2019,demo2018,demo2017,demo2016,demo2015,demo2014,demo2013,demo2012,demo2011,demo2010,demo2008,demo2006,funding2003,pledge2003,funding2004,pledge2004,funding2005,pledge2005,funding2006,pledge2006,funding2007,pledge2007,funding2008,pledge2008,funding2009,pledge2009,funding2010,pledge2010,funding2011,pledge2011,funding2012,pledge2012,funding2013,pledge2013,funding2014,pledge2014,funding2015,pledge2015,funding2016,pledge2016,funding2017,pledge2017,funding2018,pledge2018,funding2019,pledge2019,funding2020,pledge2020,gdp2003,gdp2004,gdp2005,gdp2006,gdp2007,gdp2008,gdp2009,gdp2010,gdp2011,gdp2012,gdp2013,gdp2014,gdp2015,gdp2016,gdp2017,gdp2018,gdp2019,funding_gdp2003,funding_gdp2004,funding_gdp2005,funding_gdp2006,funding_gdp2007,funding_gdp2008,funding_gdp2009,funding_gdp2010,funding_gdp2011,funding_gdp2012,funding_gdp2013,funding_gdp2014,funding_gdp2015,funding_gdp2016,funding_gdp2017,funding_gdp2018,funding_gdp2019
0,Afghanistan,AFG,-0.201360,0.120764,0.289641,0.254712,0.166455,0.315964,2.85,2.97,2.55,2.55,2.77,2.77,2.48,2.48,2.48,2.48,3.02,3.06,NaN,NaN,NaN,NaN,500000.0,NaN,NaN,NaN,NaN,NaN,24000000.0,NaN,136493.0,NaN,2379910.0,NaN,3360.0,NaN,3000.0,NaN,2000.0,NaN,NaN,NaN,1000.0,NaN,NaN,NaN,NaN,NaN,3870542.0,NaN,903177.0,NaN,NaN,NaN,4.515559e+09,5.226779e+09,6.209138e+09,6.971286e+09,9.747880e+09,1.010923e+10,1.243909e+10,1.585657e+10,1.780429e+10,2.000160e+10,2.056107e+10,2.048489e+10,1.990711e+10,1.936264e+10,2.019176e+10,1.948438e+10,1.910135e+10,NaN,NaN,0.0081,NaN,NaN,0.2374,0.0011,0.0150,0.0000,0.0000,0.0000,NaN,0.0000,NaN,NaN,0.0199,0.0047
1,Algeria,DZA,0.059815,0.391530,-0.598255,0.254901,-0.009968,-0.177371,4.01,3.50,3.56,3.56,3.95,3.83,3.83,3.83,3.44,3.44,3.32,3.17,3178000.0,NaN,NaN,NaN,14769749.0,NaN,NaN,NaN,10000.0,NaN,10000.0,NaN,76432.0,NaN,10110696.0,NaN,16010000.0,7000000.0,NaN,NaN,2000000.0,NaN,10000.0,NaN,NaN,NaN,660000.0,NaN,10000.0,NaN,10000.0,NaN,NaN,NaN,NaN,NaN,6.786485e+10,8.532477e+10,1.031984e+11,1.170254e+11,1.349773e+11,1.710006e+11,1.372107e+11,1.612078e+11,2.000195e+11,2.090603e+11,2.097548e+11,2.138088e+11,1.659794e+11,1.600338e+11,1.673894e+11,1.737580e+11,1.699882e+11,0.0047,NaN,0.0143,NaN,0.0000,0.0000,0.0001,0.0063,0.0080,NaN,0.0010,0.0000,NaN,0.0004,0.0000,0.0000,NaN
2,Argentina,ARG,-0.229308,0.041503,0.159679,-0.140457,0.027347,-0.206733,7.02,7.02,6.96,6.96,7.02,6.84,6.84,6.84,6.84,6.84,6.63,6.63,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,139000.0,NaN,31100.0,NaN,50000.0,NaN,107332.0,NaN,130826.0,NaN,165718.0,NaN,230500.0,NaN,NaN,NaN,176272.0,NaN,1344591.0,NaN,265000.0,NaN,155000.0,NaN,NaN,NaN,1.275870e+11,1.646579e+11,1.987371e+11,2.325573e+11,2.875305e+11,3.615580e+11,3.329765e+11,4.236274e+11,5.301633e+11,5.459824e+11,5.520251e+11,5.263197e+11,5.947493e+11,5.575314e+11,6.426959e+11,5.198715e+11,4.496634e+11,NaN,NaN,NaN,NaN,NaN,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,NaN,0.0000,0.0002,0.0001,0.0000
3,Australia,AUS,0.657004,0.137137,0.069660,0.022190,0.155331,0.298973,9.09,9.09,9.09,9.01,9.01,9.01,9.13,9.22,9.22,9.22,9.09,9.09,87736725.0,NaN,69076907.0,NaN,85073844.0,29859474.0,89757369.0,2262517.0,109371739.0,3817867.0,143674846.0,39494954.0,196311166.0,8733624.0,293856560.0,1923773.0,312711049.0,NaN,271004092.0,2809573.0,263194483.0,NaN,269253261.0,NaN,166236181.0,762236.0,205207886.0,2724017.0,243900706.0,2712522.0,248119692.0,NaN,310639840.0,NaN,271692972.0,NaN,4.664881e+11,6.124904e+11,6.934078e+11,7.460542e+11,8.530996e+11,1.053996e+12,9.278052e+11,1.146138e+12,1.396650e+12,1.546152e+12,1.576184e+12,1.467484e+12,1.351694e+12,1.208847e+12,1.330136e+12,1.433904e+12,1.392681e+12,0.0188,0.0113,0.0123,0.0120,0.0128,0.0136,0.0212,0.0256,0.0224,0.0175,0.0167,0.0183,0.0123,0.0170,0.0183,0.0173,0.0223
4,Austria,AUT,0.608285,-0.061829,0.161047,-0.055415,-0.035363,0.154628,8.29,8.29,8.42,8.41,8.54,8.54,8.48,8.62,8.49,8.49,8.49,8.69,3245126.0,NaN,6186042.0,NaN,18423937.0,NaN,5624017.0,765306.0,7885069.0,NaN,15280397.0,1017

### adding GDP per capita in current US dollar from WDI

<!---adding General government total expenditure as % of GDP from IMF --->

In [18]:
capita = pd.read_excel('data/gdppercapita.xls', sheet_name='Data', skiprows=3).rename(columns={"Country Code": "isocode"}).set_index('isocode')
capita_filter = capita.filter(regex='^20(10|11|12|.*[3456789]$)', axis=1).add_prefix('gdpcapita')

capita_filter.head()



,gdpcapita2003,gdpcapita2004,gdpcapita2005,gdpcapita2006,gdpcapita2007,gdpcapita2008,gdpcapita2009,gdpcapita2010,gdpcapita2011,gdpcapita2012,gdpcapita2013,gdpcapita2014,gdpcapita2015,gdpcapita2016,gdpcapita2017,gdpcapita2018,gdpcapita2019
isocode,,,,,,,,,,,,,,,,,
ABW,20833.761612,22569.974985,23300.039558,24045.272483,25835.132668,27084.703690,24630.453714,23512.602596,24985.993281,24713.698045,26189.435509,26647.938101,27980.880695,28281.350482,29007.693003,NaN,NaN
AFG,190.683814,211.382117,242.031285,263.733692,359.693238,364.660745,438.076034,543.303042,591.162759,641.871479,637.165523,613.856689,578.466353,547.228110,556.302002,524.162881,502.115487
AGO,982.960899,1255.564045,1902.422346,2599.566464,3121.995637,4080.941410,3122.780766,3587.883798,4615.468028,5100.095808,5254.882338,5408.410496,4166.979684,3506.072885,4095.812942,3289.646664,2973.591160
ALB,1846.118813,2373.579844,2673.787283,2972.743265,3595.037163,4370.540127,4114.140114,4094.350334,4437.142885,4247.629984,4413.060861,4578.631994,3952.801215,4124.055726,4531.020806,5284.380184,5352.857411
AND,32271.963902,37969.174991,40066.256919,42675.812757,47803.693608,48718.496869,43503.185516,40852.666777,43335.328862,38686.461264,39538.766722,41303.929371,35762.523074,37474.665406,38962.880354,41793.055258,40886.391165


In [19]:
df_add_capita = pd.merge(df_funding, capita_filter, how='left', on="isocode")
# df_add_capita.shape
# df_add_capita[df_add_capita.filter(regex='gdpcapita.*', axis=1).isna().all(axis=1)]





### adding government expense as % of GDP from imf


In [20]:
govexpense = pd.read_csv('data/govexpense.csv').set_index('isocode')
df_add_govexpense = pd.merge(df_add_capita, govexpense, how='left', on="isocode")
df_add_govexpense.shape
# df_add_govexpense[df_add_govexpense.filter(regex='govexpense.*', axis=1).isna().all(axis=1)]

(76, 124)

### adding region and income group data from WDI

In [21]:
region = pd.read_csv('data/WDICountry.csv').rename(columns={"Country Code": "isocode"}).set_index('isocode')
region_rename = region.rename({'Region': 'region', 'Income Group': 'income_type'}, axis=1)
region_filter = region_rename.filter(items=['region', 'income_type'], axis=1)
region_filter.head()


,region,income_type
isocode,,
ABW,Latin America & Caribbean,High income
AFG,South Asia,Low income
AGO,Sub-Saharan Africa,Lower middle income
ALB,Europe & Central Asia,Upper middle income
AND,Europe & Central Asia,High income


In [22]:
df_add_region = pd.merge(df_add_govexpense, region_filter, how='left', on="isocode")
# df_add_region.shape
# df_add_region[df_add_region.filter(items=['region', 'income_type'], axis=1).isna().any(axis=1)]

### adding population data by year from world bank

In [23]:
pop = pd.read_excel('data/Worldbank_Population_Data.xls', sheet_name='Data', skiprows=3).rename(columns={"Country Code": "isocode"}).set_index('isocode')
pop_filter = pop.filter(regex='^20(10|11|12|.*[3456789]$)', axis=1).add_prefix('pop')
pop_filter.head()

,pop2003,pop2004,pop2005,pop2006,pop2007,pop2008,pop2009,pop2010,pop2011,pop2012,pop2013,pop2014,pop2015,pop2016,pop2017,pop2018,pop2019
isocode,,,,,,,,,,,,,,,,,
ABW,97017.0,98737.0,100031.0,100834.0,101222.0,101358.0,101455.0,101669.0,102046.0,102560.0,103159.0,103774.0,104341.0,104872.0,105366.0,105845.0,106314.0
AFG,23680871.0,24726684.0,25654277.0,26433049.0,27100536.0,27722276.0,28394813.0,29185507.0,30117413.0,31161376.0,32269589.0,33370794.0,34413603.0,35383128.0,36296400.0,37172386.0,38041754.0
AGO,18121479.0,18758145.0,19433602.0,20149901.0,20905363.0,21695634.0,22514281.0,23356246.0,24220661.0,25107931.0,26015780.0,26941779.0,27884381.0,28842484.0,29816748.0,30809762.0,31825295.0
ALB,3039616.0,3026939.0,3011487.0,2992547.0,2970017.0,2947314.0,2927519.0,2913021.0,2905195.0,2900401.0,2895092.0,2889104.0,2880703.0,2876101.0,2873457.0,2866376.0,2854191.0
AND,73182.0,76244.0,78867.0,80993.0,82684.0,83862.0,84463.0,84449.0,83747.0,82427.0,80774.0,79213.0,78011.0,77297.0,77001.0,77006.0,77142.0


In [24]:
df_add_pop = pd.merge(df_add_region, pop_filter, how='left', on="isocode")
# df_add_pop.shape
# df_add_pop[df_add_pop.filter(regex='pop.*', axis=1).isna().all(axis=1)]

### adding Net Official development assistance (ODA) data in current US dollar from WDI
### notice 54/76 countries have no ODA record from 2003 ~ 2019

In [25]:
oda = pd.read_excel('data/oda.xls', sheet_name='Data', skiprows=3).rename(columns={"Country Code": "isocode"}).set_index('isocode')
oda_filter = oda.filter(regex='^20(10|11|12|.*[3456789]$)', axis=1).add_prefix('oda')
# oda_replace = oda.replace({0: np.nan})
oda_filter.head()

,oda2003,oda2004,oda2005,oda2006,oda2007,oda2008,oda2009,oda2010,oda2011,oda2012,oda2013,oda2014,oda2015,oda2016,oda2017,oda2018,oda2019
isocode,,,,,,,,,,,,,,,,,
ABW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AFG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AGO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ALB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AND,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
oda_filter['oda_int'] = oda_filter.mean(axis=1).notna().astype(int)

In [27]:
df_add_oda = pd.merge(df_add_pop, oda_filter, how='left', on="isocode")
# df_add_oda.shape
df_add_oda[df_add_oda.filter(regex='oda.*', axis=1).isna().all(axis=1)].shape

(0, 161)

In [28]:
#oda as % of GNI 
# odagni = pd.read_excel('data/oda gni.xls', sheet_name='Data', skiprows=3).rename(columns={"Country Code": "isocode"}).set_index('isocode')
# odagni_filter = odagni.filter(regex='^20(10|11|12|.*[3456789]$)', axis=1).add_prefix('odagni')
# odagni_filter.head()
# df_add_odagni = pd.merge(df_add_oda, odagni_filter, how='left', on="isocode")
# df_add_odagni.shape
# df_add_odagni[df_add_odagni.filter(regex='odagni.*', axis=1).isna().all(axis=1)].shape

### adding net official aid recieve or not in current US dollar 
### our countries of interest only appear in 2003, 2004, 2008, 2009, 2011
### notice 63/76 countries does not have any record

In [29]:

aid = pd.read_excel('data/offial aid received.xls', sheet_name='Data', skiprows=3).rename(columns={"Country Code": "isocode"}).set_index('isocode')
aid_filter = aid.filter(regex='^20(10|11|12|.*[3456789]$)', axis=1).add_prefix('aid')
aid_filter.head()

,aid2003,aid2004,aid2005,aid2006,aid2007,aid2008,aid2009,aid2010,aid2011,aid2012,aid2013,aid2014,aid2015,aid2016,aid2017,aid2018,aid2019
isocode,,,,,,,,,,,,,,,,,
ABW,7.622000e+07,-1.132000e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AFG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AGO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ALB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AND,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
aid_int = aid_filter.mean(axis=1).notna().astype(int)
df_add_aid = pd.merge(df_add_oda, aid_int.rename('aid'), how='left', on="isocode")
df_add_aid.shape


(76, 162)

In [31]:
### adding columns of GDP / population * 100 by year and compare with GDP per capita to ensure corectness
# df_capita = convert_percent('gdp','pop', df_add_aid)
# df_capita[['gdp2003_pop', 'gdpcapita2003', 'gdp2004_pop', 'gdpcapita2004']]

In [32]:
df_add_aid.head()

,country,isocode,patience,risktaking,posrecip,negrecip,altruism,trust,demo2019,demo2018,demo2017,demo2016,demo2015,demo2014,demo2013,demo2012,demo2011,demo2010,demo2008,demo2006,funding2003,pledge2003,funding2004,pledge2004,funding2005,pledge2005,funding2006,pledge2006,funding2007,pledge2007,funding2008,pledge2008,funding2009,pledge2009,funding2010,pledge2010,funding2011,pledge2011,funding2012,pledge2012,funding2013,pledge2013,funding2014,pledge2014,funding2015,pledge2015,funding2016,pledge2016,funding2017,pledge2017,funding2018,pledge2018,funding2019,pledge2019,funding2020,pledge2020,gdp2003,gdp2004,gdp2005,gdp2006,gdp2007,gdp2008,gdp2009,gdp2010,gdp2011,gdp2012,gdp2013,gdp2014,gdp2015,gdp2016,gdp2017,gdp2018,gdp2019,funding_gdp2003,funding_gdp2004,funding_gdp2005,funding_gdp2006,funding_gdp2007,funding_gdp2008,funding_gdp2009,funding_gdp2010,funding_gdp2011,funding_gdp2012,funding_gdp2013,funding_gdp2014,funding_gdp2015,funding_gdp2016,funding_gdp2017,funding_gdp2018,funding_gdp2019,gdpcapita2003,gdpcapita2004,gdpcapita2005,gdpcapita2006,gdpcapita2007,gdpcapita2008,gdpcapita2009,gdpcapita2010,gdpcapita2011,gdpcapita2012,gdpcapita2013,gdpcapita2014,gdpcapita2015,gdpcapita2016,gdpcapita2017,gdpcapita2018,gdpcapita2019,govexpense2003,govexpense2004,govexpense2005,govexpense2006,govexpense2007,govexpense2008,govexpense2009,govexpense2010,govexpense2011,govexpense2012,govexpense2013,govexpense2014,govexpense2015,govexpense2016,govexpense2017,govexpense2018,govexpense2019,region,income_type,pop2003,pop2004,pop2005,pop2006,pop2007,pop2008,pop2009,pop2010,pop2011,pop2012,pop2013,pop2014,pop2015,pop2016,pop2017,pop2018,pop2019,oda2003,oda2004,oda2005,oda2006,oda2007,oda2008,oda2009,oda2010,oda2011,oda2012,oda2013,oda2014,oda2015,oda2016,oda2017,oda2018,oda2019,oda_int,aid
0,Afghanistan,AFG,-0.201360,0.120764,0.289641,0.254712,0.166455,0.315964,2.85,2.97,2.55,2.55,2.77,2.77,2.48,2.48,2.48,2.48,3.02,3.06,NaN,NaN,NaN,NaN,500000.0,NaN,NaN,NaN,NaN,NaN,24000000.0,NaN,136493.0,NaN,2379910.0,NaN,3360.0,NaN,3000.0,NaN,2000.0,NaN,NaN,NaN,1000.0,NaN,NaN,NaN,NaN,NaN,3870542.0,NaN,903177.0,NaN,NaN,NaN,4.515559e+09,5.226779e+09,6.209138e+09,6.971286e+09,9.747880e+09,1.010923e+10,1.243909e+10,1.585657e+10,1.780429e+10,2.000160e+10,2.056107e+10,2.048489e+10,1.990711e+10,1.936264e+10,2.019176e+10,1.948438e+10,1.910135e+10,NaN,NaN,0.0081,NaN,NaN,0.2374,0.0011,0.0150,0.0000,0.0000,0.0000,NaN,0.0000,NaN,NaN,0.0199,0.0047,190.683814,211.382117,242.031285,263.733692,359.693238,364.660745,438.076034,543.303042,591.162759,641.871479,637.165523,613.856689,578.466353,547.228110,556.302002,524.162881,502.115487,11.927,15.069,15.698,18.303,21.478,20.918,21.170,20.801,21.937,25.028,24.977,25.402,25.947,28.029,27.732,28.936,27.999,South Asia,Low income,23680871.0,24726684.0,25654277.0,26433049.0,27100536.0,27722276.0,28394813.0,29185507.0,30117413.0,31161376.0,32269589.0,33370794.0,34413603.0,35383128.0,36296400.0,37172386.0,38041754.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
1,Algeria,DZA,0.059815,0.391530,-0.598255,0.254901,-0.009968,-0.177371,4.01,3.50,3.56,3.56,3.95,3.83,3.83,3.83,3.44,3.44,3.32,3.17,3178000.0,NaN,NaN,NaN,14769749.0,NaN,NaN,NaN,10000.0,NaN,10000.0,NaN,76432.0,NaN,10110696.0,NaN,16010000.0,7000000.0,NaN,NaN,2000000.0,NaN,10000.0,NaN,NaN,NaN,660000.0,NaN,10000.0,NaN,10000.0,NaN,NaN,NaN,NaN,NaN,6.786485e+10,8.532477e+10,1.031984e+11,1.170254e+11,1.349773e+11,1.710006e+11,1.372107e+11,1.612078e+11,2.000195e+11,2.090603e+11,2.097548e+11,2.138088e+11,1.659794e+11,1.600338e+11,1.673894e+11,1.737580e+11,1.699882e+11,0.0047,NaN,0.0143,NaN,0.0000,0.0000,0.0001,0.0063,0.0080,NaN,0.0010,0.0000,NaN,0.0004,0.0000,0.0000,NaN,2103.413058,2609.945608,3113.101094,3478.654871,3950.519369,4923.629265,3883.261271,4480.799271,5455.853529,5592.257099,5499.581487,5493.025590,4177.892515,3946.443977,4044.276600,4114.715061,3948.343279,32.202,30.765,27.108,28.853,33.236,37.950,42.600,37.251,40.123,43.543,36.186,40.606,45.811,41.665,39.20

### add in dummy variables indicates whether in OECD or G20 in 2019

In [33]:
OECD = ['Austria', 'Belgium', 'Czech Republic', 'Denmark', 'Estonia', 'Finland', 'France', 'Germany', 'Greece', 'Hungary', 'Iceland', 'Ireland', 'Italy', 'Latvia', 'Lithuania', 'Luxembourg', 'Netherlands', 'Norway', 'Poland', 'Portugal', 'Slovak Republic', 'Slovenia', 'Spain', 'Sweden', 'Switzerland', 'United Kingdom']
EU = [ 'Austria', 'Belgium', 'Bulgaria', 'Croatia', 'Czech Republic', 'Denmark', 'Estonia', 'Finland', 'France', 'Germany', 'Greece', 'Hungary', 'Ireland', 'Italy', 'Latvia', 'Lithuania', 'Luxembourg', 'Malta', 'Netherlands', 'Poland', 'Portugal', 'Romania', 'Slovakia', 'Slovenia', 'Spain', 'Sweden']
G20_else = ['Argentina', 'Australia', 'Brazil', 'Canada', 'China', 'France', 'Germany', 'India', 'Indonesia', 'Italy', 'Japan', 'Mexico', 'Russia', 'Saudi Arabia', 'South Africa', 'South Korea', 'Turkey', 'United Kingdom', 'United States']
G20 = G20_else + EU


In [34]:
df_add_oecd = df_add_aid.assign(oecd = df_add_aid[['country']].isin(OECD).astype('int'))
df_add_g20 = df_add_oecd.assign(g20 = df_add_aid[['country']].isin(G20).astype('int'))
df_add_g20.head()

,country,isocode,patience,risktaking,posrecip,negrecip,altruism,trust,demo2019,demo2018,demo2017,demo2016,demo2015,demo2014,demo2013,demo2012,demo2011,demo2010,demo2008,demo2006,funding2003,pledge2003,funding2004,pledge2004,funding2005,pledge2005,funding2006,pledge2006,funding2007,pledge2007,funding2008,pledge2008,funding2009,pledge2009,funding2010,pledge2010,funding2011,pledge2011,funding2012,pledge2012,funding2013,pledge2013,funding2014,pledge2014,funding2015,pledge2015,funding2016,pledge2016,funding2017,pledge2017,funding2018,pledge2018,funding2019,pledge2019,funding2020,pledge2020,gdp2003,gdp2004,gdp2005,gdp2006,gdp2007,gdp2008,gdp2009,gdp2010,gdp2011,gdp2012,gdp2013,gdp2014,gdp2015,gdp2016,gdp2017,gdp2018,gdp2019,funding_gdp2003,funding_gdp2004,funding_gdp2005,funding_gdp2006,funding_gdp2007,funding_gdp2008,funding_gdp2009,funding_gdp2010,funding_gdp2011,funding_gdp2012,funding_gdp2013,funding_gdp2014,funding_gdp2015,funding_gdp2016,funding_gdp2017,funding_gdp2018,funding_gdp2019,gdpcapita2003,gdpcapita2004,gdpcapita2005,gdpcapita2006,gdpcapita2007,gdpcapita2008,gdpcapita2009,gdpcapita2010,gdpcapita2011,gdpcapita2012,gdpcapita2013,gdpcapita2014,gdpcapita2015,gdpcapita2016,gdpcapita2017,gdpcapita2018,gdpcapita2019,govexpense2003,govexpense2004,govexpense2005,govexpense2006,govexpense2007,govexpense2008,govexpense2009,govexpense2010,govexpense2011,govexpense2012,govexpense2013,govexpense2014,govexpense2015,govexpense2016,govexpense2017,govexpense2018,govexpense2019,region,income_type,pop2003,pop2004,pop2005,pop2006,pop2007,pop2008,pop2009,pop2010,pop2011,pop2012,pop2013,pop2014,pop2015,pop2016,pop2017,pop2018,pop2019,oda2003,oda2004,oda2005,oda2006,oda2007,oda2008,oda2009,oda2010,oda2011,oda2012,oda2013,oda2014,oda2015,oda2016,oda2017,oda2018,oda2019,oda_int,aid,oecd,g20
0,Afghanistan,AFG,-0.201360,0.120764,0.289641,0.254712,0.166455,0.315964,2.85,2.97,2.55,2.55,2.77,2.77,2.48,2.48,2.48,2.48,3.02,3.06,NaN,NaN,NaN,NaN,500000.0,NaN,NaN,NaN,NaN,NaN,24000000.0,NaN,136493.0,NaN,2379910.0,NaN,3360.0,NaN,3000.0,NaN,2000.0,NaN,NaN,NaN,1000.0,NaN,NaN,NaN,NaN,NaN,3870542.0,NaN,903177.0,NaN,NaN,NaN,4.515559e+09,5.226779e+09,6.209138e+09,6.971286e+09,9.747880e+09,1.010923e+10,1.243909e+10,1.585657e+10,1.780429e+10,2.000160e+10,2.056107e+10,2.048489e+10,1.990711e+10,1.936264e+10,2.019176e+10,1.948438e+10,1.910135e+10,NaN,NaN,0.0081,NaN,NaN,0.2374,0.0011,0.0150,0.0000,0.0000,0.0000,NaN,0.0000,NaN,NaN,0.0199,0.0047,190.683814,211.382117,242.031285,263.733692,359.693238,364.660745,438.076034,543.303042,591.162759,641.871479,637.165523,613.856689,578.466353,547.228110,556.302002,524.162881,502.115487,11.927,15.069,15.698,18.303,21.478,20.918,21.170,20.801,21.937,25.028,24.977,25.402,25.947,28.029,27.732,28.936,27.999,South Asia,Low income,23680871.0,24726684.0,25654277.0,26433049.0,27100536.0,27722276.0,28394813.0,29185507.0,30117413.0,31161376.0,32269589.0,33370794.0,34413603.0,35383128.0,36296400.0,37172386.0,38041754.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
1,Algeria,DZA,0.059815,0.391530,-0.598255,0.254901,-0.009968,-0.177371,4.01,3.50,3.56,3.56,3.95,3.83,3.83,3.83,3.44,3.44,3.32,3.17,3178000.0,NaN,NaN,NaN,14769749.0,NaN,NaN,NaN,10000.0,NaN,10000.0,NaN,76432.0,NaN,10110696.0,NaN,16010000.0,7000000.0,NaN,NaN,2000000.0,NaN,10000.0,NaN,NaN,NaN,660000.0,NaN,10000.0,NaN,10000.0,NaN,NaN,NaN,NaN,NaN,6.786485e+10,8.532477e+10,1.031984e+11,1.170254e+11,1.349773e+11,1.710006e+11,1.372107e+11,1.612078e+11,2.000195e+11,2.090603e+11,2.097548e+11,2.138088e+11,1.659794e+11,1.600338e+11,1.673894e+11,1.737580e+11,1.699882e+11,0.0047,NaN,0.0143,NaN,0.0000,0.0000,0.0001,0.0063,0.0080,NaN,0.0010,0.0000,NaN,0.0004,0.0000,0.0000,NaN,2103.413058,2609.945608,3113.101094,3478.654871,3950.519369,4923.629265,3883.261271,4480.799271,5455.853529,5592.257099,5499.581487,5493.025590,4177.892515,3946.443977,4044.276600,4114.715061,3948.343279,32.202,30.765,27.108,28.853,33.236,37.950,42.600,37.251,40.123,43.543,36.186,40.606,45.811

### add Gini Index by year from World Bank
### 2019's data wasn't avalaible 
### notice 4/76 country have no observations at all

In [35]:

gni = pd.read_excel('data/GNI.xls', sheet_name='Data', skiprows=3).rename(columns={"Country Code": "isocode"}).set_index('isocode')
gni_filter = gni.filter(regex='^20(10|11|12|.*[3456789]$)', axis=1).add_prefix('gni')
gni_filter.head()

,gni2003,gni2004,gni2005,gni2006,gni2007,gni2008,gni2009,gni2010,gni2011,gni2012,gni2013,gni2014,gni2015,gni2016,gni2017,gni2018,gni2019
isocode,,,,,,,,,,,,,,,,,
ABW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AFG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AGO,NaN,NaN,NaN,NaN,NaN,42.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,51.3,NaN
ALB,NaN,NaN,30.6,NaN,NaN,30.0,NaN,NaN,NaN,29.0,NaN,34.6,32.9,33.7,33.2,NaN,NaN
AND,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
df_add_gini = pd.merge(df_add_g20, gni_filter, how='left', on="isocode")
# df_add_gini.shape
df_add_gini[df_add_gini.filter(regex='gni.*', axis=1).isna().all(axis=1)]

,country,isocode,patience,risktaking,posrecip,negrecip,altruism,trust,demo2019,demo2018,demo2017,demo2016,demo2015,demo2014,demo2013,demo2012,demo2011,demo2010,demo2008,demo2006,funding2003,pledge2003,funding2004,pledge2004,funding2005,pledge2005,funding2006,pledge2006,funding2007,pledge2007,funding2008,pledge2008,funding2009,pledge2009,funding2010,pledge2010,funding2011,pledge2011,funding2012,pledge2012,funding2013,pledge2013,funding2014,pledge2014,funding2015,pledge2015,funding2016,pledge2016,funding2017,pledge2017,funding2018,pledge2018,funding2019,pledge2019,funding2020,pledge2020,gdp2003,gdp2004,gdp2005,gdp2006,gdp2007,gdp2008,gdp2009,gdp2010,gdp2011,gdp2012,gdp2013,gdp2014,gdp2015,gdp2016,gdp2017,gdp2018,gdp2019,funding_gdp2003,funding_gdp2004,funding_gdp2005,funding_gdp2006,funding_gdp2007,funding_gdp2008,funding_gdp2009,funding_gdp2010,funding_gdp2011,funding_gdp2012,funding_gdp2013,funding_gdp2014,funding_gdp2015,funding_gdp2016,funding_gdp2017,funding_gdp2018,funding_gdp2019,gdpcapita2003,gdpcapita2004,gdpcapita2005,gdpcapita2006,gdpcapita2007,gdpcapita2008,gdpcapita2009,gdpcapita2010,gdpcapita2011,gdpcapita2012,gdpcapita2013,gdpcapita2014,gdpcapita2015,gdpcapita2016,gdpcapita2017,gdpcapita2018,gdpcapita2019,govexpense2003,govexpense2004,govexpense2005,govexpense2006,govexpense2007,govexpense2008,govexpense2009,govexpense2010,govexpense2011,govexpense2012,govexpense2013,govexpense2014,govexpense2015,govexpense2016,govexpense2017,govexpense2018,govexpense2019,region,income_type,pop2003,pop2004,pop2005,pop2006,pop2007,pop2008,pop2009,pop2010,pop2011,pop2012,pop2013,pop2014,pop2015,pop2016,pop2017,pop2018,pop2019,oda2003,oda2004,oda2005,oda2006,oda2007,oda2008,oda2009,oda2010,oda2011,oda2012,oda2013,oda2014,oda2015,oda2016,oda2017,oda2018,oda2019,oda_int,aid,oecd,g20,gni2003,gni2004,gni2005,gni2006,gni2007,gni2008,gni2009,gni2010,gni2011,gni2012,gni2013,gni2014,gni2015,gni2016,gni2017,gni2018,gni2019
0,Afghanistan,AFG,-0.201360,0.120764,0.289641,0.254712,0.166455,0.315964,2.85,2.97,2.55,2.55,2.77,2.77,2.48,2.48,2.48,2.48,3.02,3.06,NaN,NaN,NaN,NaN,500000.0,NaN,NaN,NaN,NaN,NaN,24000000.0,NaN,136493.0,NaN,2379910.0,NaN,3360.0,NaN,3000.0,NaN,2000.0,NaN,NaN,NaN,1000.0,NaN,NaN,NaN,NaN,NaN,3.870542e+06,NaN,9.031770e+05,NaN,NaN,NaN,4.515559e+09,5.226779e+09,6.209138e+09,6.971286e+09,9.747880e+09,1.010923e+10,1.243909e+10,1.585657e+10,1.780429e+10,2.000160e+10,2.056107e+10,2.048489e+10,1.990711e+10,1.936264e+10,2.019176e+10,1.948438e+10,1.910135e+10,NaN,NaN,0.0081,NaN,NaN,0.2374,0.0011,0.0150,0.0000,0.000,0.0000,NaN,0.0000,NaN,NaN,0.0199,0.0047,190.683814,211.382117,242.031285,263.733692,359.693238,364.660745,438.076034,543.303042,591.162759,641.871479,637.165523,613.856689,578.466353,547.228110,556.302002,524.162881,502.115487,11.927,15.069,15.698,18.303,21.478,20.918,21.170,20.801,21.937,25.028,24.977,25.402,25.947,28.029,27.732,28.936,27.999,South Asia,Low income,23680871.0,24726684.0,25654277.0,26433049.0,27100536.0,27722276.0,28394813.0,29185507.0,30117413.0,31161376.0,32269589.0,33370794.0,34413603.0,35383128.0,36296400.0,37172386.0,38041754.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,Cambodia,KHM,-0.119899,-0.404640,0.171683,-0.062052,-0.216157,-0.079135,3.53,3.59,3.63,4.27,4.27,4.78,4.60,4.96,4.87,4.87,4.87,4.77,NaN,NaN,NaN,NaN,10000.0,NaN,NaN,NaN,NaN,NaN,310000.0,NaN,1554.0,NaN,60000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20000.0,NaN,4.658247e+09,5.337833e+09,6.293046e+09,7.274596e+09,8.639236e+09,1.035191e+10,1.040185e+10,1.124228e+10,1.282954e+10,1.405444e+10,1.522799e+10,1.670261e+10,1.804995e+10,2.001675e+10,2.217720e+10,2.457175e+10,2.708939e+10,NaN,NaN,0.0002,NaN,NaN,0.0030,0.0000,0.0005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,362.335708,408.513826,474.111228,539.750169,631.524842,745.609181,738.054784,785.502283,882.275493,950.880346,1013.420401,1093.496191,1162.904920,1269.5

### output

In [38]:
df_add_gini.to_csv('data/result.csv', index=False) 